In [1]:
import pandas as pd
import numpy as np

np.float_ = np.float64

from prophet import Prophet
import requests
from datetime import datetime, timedelta
import duckdb as db

m = Prophet()
conn = db.connect("file.db")
conn.sql("""
    CREATE SEQUENCE seq_id START 1;
    CREATE TABLE contract_call (id INTEGER NOT NULL PRIMARY KEY, charged_tx_fee INTEGER, time_tx_occured STRING)
""")

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Get data

end_time = datetime.now()
print(end_time.timestamp())
start_time = (end_time - timedelta(seconds=3600.0)).timestamp()
print(start_time)

transaction_types = ["CONTRACTCALL"]

from typing import Union
from pprint import pprint
from uuid import uuid4

def get_data(start_time: float | int, limit: int, transaction_type: str, order: Union["asc", "desc"]):
    url = f"https://testnet.mirrornode.hedera.com/api/v1/transactions?limit={limit}&order={order}&timestamp=gte%3A{start_time}&transactiontype={transaction_type}"
    data = requests.get(url).json()
    transactions = data["transactions"]
    new_transactions: list[dict | None] = []
    for transaction in transactions:
        new_data = {}
        if not transaction["scheduled"] and transaction["result"] == "SUCCESS":
            _, seconds, nanoseconds = transaction["transaction_id"].split("-")
            # new_data["id"] = str(uuid4())
            new_data["charged_tx_fee"] = transaction["charged_tx_fee"]
            new_data["time_tx_occured"] = datetime.fromtimestamp(float(f"{seconds}.{nanoseconds}")).isoformat()

            new_transactions.append(new_data)
    return new_transactions


transactions: list[dict | None] = get_data(start_time, 100, transaction_types[0], "asc")
pprint(transactions)

1723976482.998212
1723972882.998212
[{'charged_tx_fee': 130400000, 'time_tx_occured': '2024-08-18T09:36:40.882090'},
 {'charged_tx_fee': 521600000, 'time_tx_occured': '2024-08-18T09:51:48.407779'},
 {'charged_tx_fee': 130400000, 'time_tx_occured': '2024-08-18T09:54:20.483274'},
 {'charged_tx_fee': 521600000, 'time_tx_occured': '2024-08-18T09:54:29.045515'},
 {'charged_tx_fee': 521600000, 'time_tx_occured': '2024-08-18T09:54:31.292564'},
 {'charged_tx_fee': 130400000, 'time_tx_occured': '2024-08-18T09:55:48.866523'},
 {'charged_tx_fee': 130400000, 'time_tx_occured': '2024-08-18T09:57:23.039107'},
 {'charged_tx_fee': 130400000, 'time_tx_occured': '2024-08-18T09:57:54.725575'},
 {'charged_tx_fee': 130400000, 'time_tx_occured': '2024-08-18T09:59:43.452545'},
 {'charged_tx_fee': 129600000, 'time_tx_occured': '2024-08-18T10:00:34.073437'},
 {'charged_tx_fee': 129600000, 'time_tx_occured': '2024-08-18T10:17:51.027050'}]


In [5]:
for transaction in transactions:
    conn.sql(f"""
        INSERT INTO contract_call VALUES (NEXTVAL('seq_id'), {transaction["charged_tx_fee"]}, {transaction["time_tx_occured"]})
    """)

db.sql("""
    SELECT * FROM contract_call
""")

ParserException: Parser Error: syntax error at or near "T09"

In [63]:
new_df = df.drop(["bytes", "entity_id", "max_fee", "memo_base64", "name", "node", "nonce", "parent_consensus_timestamp", "consensus_timestamp", "transaction_hash", "valid_duration_seconds", "valid_start_timestamp", "nft_transfers", "token_transfers", "staking_reward_transfers"], axis="columns")
new_df.tail()

,charged_tx_fee,result,scheduled,transaction_id,transfers
26,158400000,SUCCESS,False,0.0.2119-1723892758-084217830,"[{'account': '0.0.98', 'amount': 158400000, 'i..."
27,0,SUCCESS,False,0.0.2119-1723892758-084217830,[]
28,0,SUCCESS,False,0.0.2119-1723892758-084217830,[]
29,0,SUCCESS,False,0.0.2119-1723892758-084217830,[]
30,0,SUCCESS,False,0.0.2119-1723892758-084217830,[]


In [71]:
new_df.head()

,charged_tx_fee,result,scheduled,transaction_id,transfers
0,0,REVERTED_SUCCESS,False,0.0.902-1723889449-455599807,[]
1,0,REVERTED_SUCCESS,False,0.0.902-1723889449-455599807,[]
2,0,REVERTED_SUCCESS,False,0.0.902-1723889449-455599807,[]
3,0,REVERTED_SUCCESS,False,0.0.902-1723889516-033567002,[]
4,0,REVERTED_SUCCESS,False,0.0.902-1723890066-690469977,[]


In [70]:
pd.json_normalize(new_df, record_path=[4], meta=["charged_tx_fee", "result", "scheduled", "transaction_id", "transfers"])

TypeError: charged_tx_fee has non list value g for path 4. Must be list or null.